In [26]:
import sys

from itertools import chain, combinations
from collections import defaultdict
from optparse import OptionParser

In [27]:
transactions = [
    ["I1", "I2", "I5"],
    ["I2", "I4"],
    ["I2", "I3"],
    ["I1", "I2", "I4"],
    ["I1", "I3"],
    ["I2", "I3"],
    ["I1", "I3"],
    ["I1", "I2", "I3", "I5"],
    ["I1", "I2", "I3"],
]

In [28]:
def subsets(arr):
    """ Returns non empty subsets of arr"""
    return chain(*[combinations(arr, i + 1) for i, a in enumerate(arr)])


def returnItemsWithMinSupport(itemSet, transactionList, minSupport, freqSet):
    """calculates the support for items in the itemSet and returns a subset
    of the itemSet each of whose elements satisfies the minimum support"""
    _itemSet = set()
    localSet = defaultdict(int)

    for item in itemSet:
        for transaction in transactionList:
            if item.issubset(transaction):
                freqSet[item] += 1
                localSet[item] += 1

    for item, count in localSet.items():
        support = float(count) / len(transactionList)

        if support >= minSupport:
            _itemSet.add(item)

    return _itemSet

def joinSet(itemSet, length):
    """Join a set with itself and returns the n-element itemsets"""
    return set(
        [i.union(j) for i in itemSet for j in itemSet if len(i.union(j)) == length]
    )


def getItemSetTransactionList(data_iterator):
    transactionList = list()
    itemSet = set()
    for record in data_iterator:
        transaction = frozenset(record)
        transactionList.append(transaction)
        for item in transaction:
            itemSet.add(frozenset([item]))  # Generate 1-itemSets
    return itemSet, transactionList

#4-b)
def runApriori(data_iter, minSupport, minConfidence):
    """
    run the apriori algorithm. data_iter is a record iterator
    Return both:
     - items (tuple, support)
     - rules ((pretuple, posttuple), confidence)
    """
    itemSet, transactionList = getItemSetTransactionList(data_iter)
    print(itemSet)
    print(transactionList)
    freqSet = defaultdict(int)
    largeSet = dict()
    # Global dictionary which stores (key=n-itemSets,value=support)
    # which satisfy minSupport

    assocRules = dict()
    # Dictionary which stores Association Rules

    oneCSet = returnItemsWithMinSupport(itemSet, transactionList, minSupport, freqSet)

    currentLSet = oneCSet
    k = 2
    while currentLSet != set([]):
        largeSet[k - 1] = currentLSet
        currentLSet = joinSet(currentLSet, k)
        currentCSet = returnItemsWithMinSupport(
            currentLSet, transactionList, minSupport, freqSet
        )
        currentLSet = currentCSet
        k = k + 1

    def getSupport(item):
        """local function which Returns the support of an item"""
        return float(freqSet[item]) / len(transactionList)

    toRetItems = []
    for key, value in largeSet.items():
        toRetItems.extend([(tuple(item), getSupport(item)) for item in value])

    toRetRules = []
    for key, value in list(largeSet.items())[1:]:
        for item in value:
            _subsets = map(frozenset, [x for x in subsets(item)])
            for element in _subsets:
                remain = item.difference(element)
                if len(remain) > 0:
                    confidence = getSupport(item) / getSupport(element)
                    if confidence >= minConfidence:
                        toRetRules.append(((tuple(element), tuple(remain)), confidence))
    return toRetItems, toRetRules


def printResults(items, rules):
    """prints the generated itemsets sorted by support and the confidence rules sorted by confidence"""
    for item, support in sorted(items, key=lambda x: x[1]):
        print("item: %s , %.3f" % (str(item), support))
    print("\n------------------------ RULES:")
    for rule, confidence in sorted(rules, key=lambda x: x[1]):
        pre, post = rule
        print("Rule: %s ==> %s , %.3f" % (str(pre), str(post), confidence))
#4-A)

def supAndConf(items, rules):
    """prints the generated itemsets sorted by support and the confidence rules sorted by confidence"""
    i, r = [], []
    for item, support in sorted(items, key=lambda x: x[1]):
        x = "Support item: %s , %.3f" % (str(item), support)
        i.append(x)

    for rule, confidence in sorted(rules, key=lambda x: x[1]):
        pre, post = rule
        x = "Confidence Rule: %s ==> %s , %.3f" % (str(pre), str(post), confidence)
        r.append(x)

    return i, r

In [29]:
#4-C)

items, rules = runApriori(transactions, 0.15, 0.6)
#Printing results of
printResults(items, rules)
supAndConf(items,rules)

{frozenset({'I3'}), frozenset({'I1'}), frozenset({'I5'}), frozenset({'I4'}), frozenset({'I2'})}
[frozenset({'I2', 'I1', 'I5'}), frozenset({'I2', 'I4'}), frozenset({'I2', 'I3'}), frozenset({'I2', 'I4', 'I1'}), frozenset({'I1', 'I3'}), frozenset({'I2', 'I3'}), frozenset({'I1', 'I3'}), frozenset({'I2', 'I1', 'I5', 'I3'}), frozenset({'I2', 'I1', 'I3'})]
item: ('I5',) , 0.222
item: ('I4',) , 0.222
item: ('I2', 'I5') , 0.222
item: ('I2', 'I4') , 0.222
item: ('I1', 'I5') , 0.222
item: ('I2', 'I1', 'I3') , 0.222
item: ('I2', 'I1', 'I5') , 0.222
item: ('I2', 'I3') , 0.444
item: ('I2', 'I1') , 0.444
item: ('I1', 'I3') , 0.444
item: ('I3',) , 0.667
item: ('I1',) , 0.667
item: ('I2',) , 0.778

------------------------ RULES:
Rule: ('I3',) ==> ('I2',) , 0.667
Rule: ('I1',) ==> ('I2',) , 0.667
Rule: ('I1',) ==> ('I3',) , 0.667
Rule: ('I3',) ==> ('I1',) , 0.667
Rule: ('I5',) ==> ('I2',) , 1.000
Rule: ('I4',) ==> ('I2',) , 1.000
Rule: ('I5',) ==> ('I1',) , 1.000
Rule: ('I5',) ==> ('I2', 'I1') , 1.000


(["Support item: ('I5',) , 0.222",
  "Support item: ('I4',) , 0.222",
  "Support item: ('I2', 'I5') , 0.222",
  "Support item: ('I2', 'I4') , 0.222",
  "Support item: ('I1', 'I5') , 0.222",
  "Support item: ('I2', 'I1', 'I3') , 0.222",
  "Support item: ('I2', 'I1', 'I5') , 0.222",
  "Support item: ('I2', 'I3') , 0.444",
  "Support item: ('I2', 'I1') , 0.444",
  "Support item: ('I1', 'I3') , 0.444",
  "Support item: ('I3',) , 0.667",
  "Support item: ('I1',) , 0.667",
  "Support item: ('I2',) , 0.778"],
 ["Confidence Rule: ('I3',) ==> ('I2',) , 0.667",
  "Confidence Rule: ('I1',) ==> ('I2',) , 0.667",
  "Confidence Rule: ('I1',) ==> ('I3',) , 0.667",
  "Confidence Rule: ('I3',) ==> ('I1',) , 0.667",
  "Confidence Rule: ('I5',) ==> ('I2',) , 1.000",
  "Confidence Rule: ('I4',) ==> ('I2',) , 1.000",
  "Confidence Rule: ('I5',) ==> ('I1',) , 1.000",
  "Confidence Rule: ('I5',) ==> ('I2', 'I1') , 1.000",
  "Confidence Rule: ('I2', 'I5') ==> ('I1',) , 1.000",
  "Confidence Rule: ('I1', 'I5')